# MNIST CNN Example
Adapted from [TensorFlow: Deep MNIST for Experts](https://www.tensorflow.org/tutorials/mnist/pros/index.html).

In [ ]:
import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [ ]:
BATCH_SIZE = 50
MAX_STEPS = 2000
DROPOUT = 0.5

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
x = tf.placeholder(tf.float32, [None, 784], "X")
y_ = tf.placeholder(tf.float32, [None, 10], "Y_")

x_image = tf.reshape(x, [-1,28,28,1])

# Conv1
conv1 = tt.network.conv2d(x_image,
                          32, 5, 5, 1, 1,
                          stddev=0.02, bias=0.1,
                          activation=tf.nn.relu, name="Conv1")
h_pool1 = tt.network.max_pool(conv1, 2, 2, 2, 2, name="MaxPool1")

# Conv2
conv2 = tt.network.conv2d(h_pool1,
                          64, 5, 5, 1, 1,
                          stddev=0.02, bias=0.1,
                          activation=tf.nn.relu, name="Conv2")
h_pool2 = tt.network.max_pool(conv2, 2, 2, 2, 2, name="MaxPool2")

# FC
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tt.network.linear(h_pool2_flat, 1024,
                          "FC", stddev=0.02, bias=0.1,
                          activation=tf.nn.relu)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output
y_conv=tf.nn.softmax(tt.network.linear(h_fc1_drop, 10,
                                       "Output", stddev=0.02, bias=0.1))

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

In [ ]:
tt.visualization.show_graph(sess.graph_def)

In [ ]:
for i in range(MAX_STEPS):
    batch = mnist.train.next_batch(BATCH_SIZE)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
                                       x:batch[0],
                                       y_: batch[1],
                                       keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: DROPOUT})

print("test accuracy %g" % accuracy.eval(feed_dict={
                                         x: mnist.test.images[:1000],
                                         y_: mnist.test.labels[:1000],
                                         keep_prob: 1.0}))